In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from pathlib import Path

from SJ_Haar_CNV import base_search
from tests import test_data

In [2]:
X = np.concatenate([np.repeat (5,13), np.repeat (7,19), np.repeat(5,10)])
X

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [3]:
first_break = base_search.choose_break(X,0,X.size)
first_break

13

In [4]:
second_break_one = base_search.choose_break(X,0,first_break)
second_break_one

1

In [5]:
second_break_two = base_search.choose_break(X,first_break+1,X.size)
second_break_two

15

In [22]:
# I'm trying to manually figure out what's going on
l = X.size
s = 13
e = l
l = e - s
matrix = base_search.haar_matrix(s,e,l)
scores = np.abs(np.matmul(matrix,X[s:e]))
best_options = np.argwhere(scores == np.nanmax(scores)).flatten() 
best_options[np.abs(best_options - l //2).argmin()]

0

In [28]:
X[s:e].shape

(29,)

In [27]:
matrix.shape

(28, 29)

In [9]:
scores

array([3.27326835, 2.55883158, 2.11288564, 1.79284291, 1.5430335 ,
       1.33630621, 1.15727512, 0.99602384, 0.84515425, 0.69786316,
       0.54554473, 0.37062466])

In [10]:
best_options

array([1], dtype=int64)